In [237]:
from src.features.build_features import handle_zeros, resample_KL
from src.visualization.visualize import plot_divergence
from src.data.make_dataset import preprocess
from pathlib import Path
import pandas as pd
import numpy as np


In [238]:
FILE_NAME = 'uniform_date.csv'
PATH = Path('../data/raw')
cycle3 = pd.read_csv(PATH / FILE_NAME)
processed = preprocess(cycle3)
processed = handle_zeros(processed)

/Users/zarak/.local/share/virtualenvs/focus_plots-vLdWlo3v/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



> For the group of timestamps that you are considering for each bubble, we want to calculate the mean for each bin mu_b where b is the bin. Then for each forecast f_t_b, where t is the index of the timestamp, we are going to calculate the KL-divergence
K_t = sum(mu_b * log2(f_t_b / mu_b)) along the b axis
Finally we are going to calculate the mean forKL, mean(KL_t) along the t axis.

**So we will have to compute means for the group, during that interval, then we will compute the divergences from that mean for each forecast in the time interval, and then we will average the divergences.**

In [257]:
processed.head(3)

,Forecaster,Forecaster.ID,Question,Forecast,TeamName,Commuity.ID,Ordered.Bin.Number,Number.of.Bins,True.Answer,Weight,WRPS,CrossEntropy,FairSkill,SWRPS,Final,Initial,Process.Gain,Tvalue
Uniform Date Format,,,,,,,,,,,,,,,,,,
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.7,Kiwi,1,1,2,0.60,1.0,0.01,-0.695,-0.002,0.10,0,1,0.15,43780.91706
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.3,Kiwi,1,2,2,0.40,1.0,0.01,-0.695,-0.002,0.10,0,1,0.15,43780.91706
2019-11-11 22:02:20,Dimitry Sarin,71,CFF 308.1,0.3,Kiwi,1,2,2,0.93,1.0,0.3969,-1.143,-0.450,0.63,0,1,-0.37,43780.91829


- The KL-divergence I had in mind is different. First we compute the average of the forecasts in that time window, then we compute for each forecaster the divergence of that forecaster from that time window average. I also prefer to vectorize that operation as I did in DTC, because I get a feel for the dimensions involved.
- vector{Ave_i} dot.product ( log2( {matrix forecasters of shape (m,n)} / vector{Ave_i} ) )
- The division is done by bin with broadcast to all n forecasters. Then when you have all the KL_divergences for each forecaster, you average them to obtain the bubble size for that time window.
- The number you calculated, I believe is sort of an entropy for the average.


Notes:
1. $m$ is the number of bins
2. $n$ is the number of forecasters
3. $i$ is the index of the timestamp

In [289]:
def kullback_leibler(forecasts, average):
    print("forecasts", forecasts)
    print("average", average)
    return -average @ (np.log2(forecasts) - np.log2(average.T))

In [314]:
def KL_apply(question):
    if not question.empty:
        average_forecast = question.groupby(
            ['Ordered.Bin.Number']
        ).Forecast.mean().reset_index()
        forecasts = question.pivot_table(
            values='Forecast',
            index='Ordered.Bin.Number',
            columns='Forecaster.ID'
        ).values
        question_avg = average_forecast.Forecast.values.reshape(1, -1)
        divergences = kullback_leibler(forecasts, question_avg)
        print('divs', divergences)
        return np.mean(divergences)

In [315]:
processed.Question.unique()

array(['CFF 307.1', 'CFF 308.1', 'CFF 308.3', 'CFF 308.4', 'CFF 307.3',
       'CFF 307.4', 'CFF 307.2', 'CFF 301.1', 'CFF 301.2', 'CFF 301.4',
       'CFF 301.3', 'CFF 304.1', 'CFF 304.2', 'CFF 304.4', 'CFF 304.3',
       'CFF 308.2'], dtype=object)

In [316]:
processed.head(2)

,Forecaster,Forecaster.ID,Question,Forecast,TeamName,Commuity.ID,Ordered.Bin.Number,Number.of.Bins,True.Answer,Weight,WRPS,CrossEntropy,FairSkill,SWRPS,Final,Initial,Process.Gain,Tvalue
Uniform Date Format,,,,,,,,,,,,,,,,,,
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.7,Kiwi,1,1,2,0.6,1.0,0.01,-0.695,-0.002,0.1,0,1,0.15,43780.91706
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.3,Kiwi,1,2,2,0.4,1.0,0.01,-0.695,-0.002,0.1,0,1,0.15,43780.91706


In [317]:
question = 'CFF 308.2'
resampling_period = 'H'

In [319]:
processed.query("Question == @question").reset_index().groupby(pd.Grouper(
    freq='H',
    label='right',
    key='Uniform Date Format',
)).apply(KL_apply).head(60)

forecasts [[0.02]
 [0.25]
 [0.59]
 [0.14]]
average [[0.02 0.25 0.59 0.14]]
divs [[0.]]
forecasts [[0.005]
 [0.05 ]
 [0.85 ]
 [0.1  ]]
average [[0.005 0.05  0.85  0.1  ]]
divs [[0.]]
forecasts [[0.04]
 [0.22]
 [0.5 ]
 [0.24]]
average [[0.04 0.22 0.5  0.24]]
divs [[0.]]
forecasts [[0.3  ]
 [0.6  ]
 [0.1  ]
 [0.005]]
average [[0.3   0.6   0.1   0.005]]
divs [[0.]]
forecasts [[0.05   0.56  ]
 [0.2375 0.39  ]
 [0.575  0.05  ]
 [0.1375 0.005 ]]
average [[0.26857143 0.30285714 0.35       0.08071429]]
divs [[0.44487859 0.91125484]]
forecasts [[0.005]
 [0.29 ]
 [0.65 ]
 [0.06 ]]
average [[0.005 0.29  0.65  0.06 ]]
divs [[0.]]
forecasts [[0.005]
 [0.29 ]
 [0.64 ]
 [0.07 ]]
average [[0.005 0.29  0.64  0.07 ]]
divs [[0.]]
forecasts [[0.17]
 [0.54]
 [0.25]
 [0.04]]
average [[0.17 0.54 0.25 0.04]]
divs [[0.]]
forecasts [[0.04]
 [0.11]
 [0.55]
 [0.3 ]]
average [[0.04 0.11 0.55 0.3 ]]
divs [[0.]]
forecasts [[0.25 0.05 0.25 0.05]
 [0.25 0.2  0.25 0.38]
 [0.25 0.65 0.25 0.41]
 [0.25 0.1  0.25 0.16]]
ave

forecasts [[0.01 0.04]
 [0.18 0.19]
 [0.59 0.59]
 [0.22 0.18]]
average [[0.025 0.185 0.59  0.2  ]]
divs [[0.01286024 0.0063311 ]]
forecasts [[0.005 0.1   0.06 ]
 [0.05  0.25  0.22 ]
 [0.8   0.45  0.5  ]
 [0.15  0.2   0.22 ]]
average [[0.055      0.17333333 0.58333333 0.19      ]]
divs [[0.30013457 0.06531428 0.02302017]]
forecasts [[0.05  0.06 ]
 [0.425 0.22 ]
 [0.425 0.5  ]
 [0.1   0.22 ]]
average [[0.05333333 0.35666667 0.45       0.14      ]]
divs [[0.0198374  0.07986786]]
forecasts [[0.01  0.005]
 [0.15  0.3  ]
 [0.62  0.5  ]
 [0.22  0.2  ]]
average [[0.0075 0.225  0.56   0.21  ]]
divs [[0.03217862 0.01734483]]
forecasts [[0.05 0.05]
 [0.25 0.25]
 [0.55 0.46]
 [0.15 0.24]]
average [[0.05  0.25  0.505 0.195]]
divs [[0.01161991 0.00958376]]
forecasts [[0.02       0.02      ]
 [0.33333333 0.37      ]
 [0.49666667 0.51      ]
 [0.15       0.1       ]]
average [[0.02   0.3425 0.5    0.1375]]
divs [[0.00096949 0.01072548]]
forecasts [[0.02]
 [0.1 ]
 [0.47]
 [0.41]]
average [[0.02 0.1  0.

Uniform Date Format
2019-11-12 01:00:00    0.000000
2019-11-12 02:00:00         NaN
2019-11-12 03:00:00    0.000000
2019-11-12 04:00:00         NaN
2019-11-12 05:00:00         NaN
2019-11-12 06:00:00         NaN
2019-11-12 07:00:00         NaN
2019-11-12 08:00:00         NaN
2019-11-12 09:00:00         NaN
2019-11-12 10:00:00    0.000000
2019-11-12 11:00:00         NaN
2019-11-12 12:00:00         NaN
2019-11-12 13:00:00    0.000000
2019-11-12 14:00:00    0.678067
2019-11-12 15:00:00         NaN
2019-11-12 16:00:00    0.000000
2019-11-12 17:00:00    0.000000
2019-11-12 18:00:00    0.000000
2019-11-12 19:00:00         NaN
2019-11-12 20:00:00    0.000000
2019-11-12 21:00:00         NaN
2019-11-12 22:00:00    0.138895
2019-11-12 23:00:00    0.000000
2019-11-13 00:00:00         NaN
2019-11-13 01:00:00         NaN
2019-11-13 02:00:00    0.000000
2019-11-13 03:00:00    0.128195
2019-11-13 04:00:00         NaN
2019-11-13 05:00:00    0.030408
2019-11-13 06:00:00    0.000000
2019-11-13 07:00:00 